In [56]:
import pandas as pd
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns

In [57]:
train = pd.read_csv('./train.csv')

In [58]:
train['Age'] = train['Age'].fillna(train['Age'].mean())

In [59]:
train.loc[train['Age']<10, 'Age'] = 0
train.loc[(train['Age']>=10)&(train['Age']<20), 'Age'] = 1
train.loc[(train['Age']>=20)&(train['Age']<30), 'Age'] = 2
train.loc[(train['Age']>=30)&(train['Age']<40), 'Age'] = 3
train.loc[(train['Age']>=40)&(train['Age']<50), 'Age'] = 4
train.loc[train['Age']>=50, 'Age'] = 5

In [60]:
train['Fam'] = train['SibSp']+train['Parch']

In [61]:
print(train.count())
print(train.loc[train['Embarked'] == 'C', 'Embarked'].count())
print(train.loc[train['Embarked'] == 'S', 'Embarked'].count())
print(train.loc[train['Embarked'] == 'Q', 'Embarked'].count())
train['Embarked'] = train['Embarked'].fillna('S')

PassengerId    891
Survived       891
Pclass         891
Name           891
Sex            891
Age            891
SibSp          891
Parch          891
Ticket         891
Fare           891
Cabin          204
Embarked       889
Fam            891
dtype: int64
168
644
77


In [62]:
train.loc[train['Embarked']=='C', 'Embarked'] =0
train.loc[train['Embarked']=='S', 'Embarked'] =1
train.loc[train['Embarked']=='Q', 'Embarked'] =2

In [63]:
train.loc[train['Sex']=='male', 'Sex'] =0
train.loc[train['Sex']=='female', 'Sex'] =1

In [64]:
x_train = train[['Pclass', 'Sex', 'Age', 'Fam','Embarked']]
y_train = train[['Survived']]

In [65]:
from sklearn.tree import DecisionTreeClassifier
x_train.isnull().sum()

Pclass      0
Sex         0
Age         0
Fam         0
Embarked    0
dtype: int64

In [66]:
x_valid1 = x_train[0:100]
y_valid1 = y_train[0:100]

x_train1 = x_train[100:]
y_train1 = y_train[100:]

tree1 = DecisionTreeClassifier()
tree1.fit(x_train1, y_train1)
print(tree1.score(x_train1, y_train1))
print(tree1.score(x_valid1, y_valid1))

0.8735777496839444
0.76


In [67]:
x_valid2 = x_train[100:]
y_valid2 = y_train[100:]

x_train2 = x_train[0:100]
y_train2 = y_train[0:100]

tree2 = DecisionTreeClassifier()
tree2.fit(x_train2, y_train2)
print(tree2.score(x_train2, y_train2))
print(tree2.score(x_valid2, y_valid2))

0.91
0.7825537294563844


In [68]:
x_valid3 = x_train[100:200]
y_valid3 = y_train[100:200]

x_train3 = pd.concat([x_train[0:100], x_train[200:]])
y_train3 = pd.concat([y_train[0:100], y_train[200:]])

tree3 = DecisionTreeClassifier()
tree3.fit(x_train3, y_train3)
print(tree3.score(x_train3, y_train3))
print(tree3.score(x_valid3, y_valid3))

0.8647281921618205
0.82


### test

In [69]:
test = pd.read_csv('./test.csv')

In [70]:
test.isnull().sum()

PassengerId      0
Pclass           0
Name             0
Sex              0
Age             86
SibSp            0
Parch            0
Ticket           0
Fare             1
Cabin          327
Embarked         0
dtype: int64

In [71]:
test['Age'] = test['Age'].fillna(test['Age'].mean())
test.loc[test['Age']<10, 'Age'] =0
test.loc[(test['Age']>=10)&(test['Age']<20), 'Age'] =1
test.loc[(test['Age']>=20)&(test['Age']<30), 'Age'] =2
test.loc[(test['Age']>=30)&(test['Age']<40), 'Age'] =3
test.loc[(test['Age']>=40)&(test['Age']<50), 'Age'] =4
test.loc[test['Age']>=50, 'Age'] =5

In [72]:
test['Fam'] = test['SibSp']+test['Parch']

In [73]:
test.loc[test['Embarked']=='C', 'Embarked'] =0
test.loc[test['Embarked']=='S', 'Embarked'] =1
test.loc[test['Embarked']=='Q', 'Embarked'] =2

In [74]:
test.loc[test['Sex']=='male', 'Sex'] =0
test.loc[test['Sex']=='female', 'Sex'] =1

In [75]:
x_test = test[['Pclass', 'Sex', 'Age', 'Fam','Embarked']]

In [76]:
prediction1 = tree1.predict(x_test)
prediction2 = tree2.predict(x_test)
prediction3 = tree3.predict(x_test)

In [77]:
ensemble = (prediction1 + prediction2 +prediction3)/3
ensemble[ensemble >0.5] = 1
ensemble[ensemble <=0.5] = 0

ensemble = ensemble.astype(int)

In [79]:
submit = pd.DataFrame({
    'PassengerId':test['PassengerId'],
    'Survived' : ensemble
})
submit.to_csv('submit1.csv', index=False)